In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
# conexao sql3
conn = sql.connect('db.sqlite3')

In [3]:
# professores
sql_professor = '''
SELECT G.NAME, UG.ID, P.NOME_COMPLETO
  FROM AUTH_GROUP G
       INNER JOIN AUTH_USER_GROUPS UG ON G.ID = UG.GROUP_ID
       INNER JOIN USUARIOS_PERFIL P ON P.ID = UG.ID
 WHERE G.ID = 2
 '''
professores = pd.read_sql(sql_professor, conn)
professores

,name,id,nome_completo
0,Professor,2,Daine Rocha
1,Professor,3,Alex Rocha
2,Professor,9,Vanessa Silva


In [4]:
# alunos
sql_alunos = '''
SELECT G.NAME, UG.ID, P.NOME_COMPLETO
  FROM AUTH_GROUP G
       INNER JOIN AUTH_USER_GROUPS UG ON G.ID = UG.GROUP_ID
       INNER JOIN USUARIOS_PERFIL P ON P.ID = UG.ID
 WHERE G.ID = 3
'''
alunos = pd.read_sql(sql_alunos, conn)
alunos

,name,id,nome_completo
0,Aluno,1,Mario Santos
1,Aluno,5,Roberto Rocha
2,Aluno,6,Ana Pereira
3,Aluno,7,Maria Barros
4,Aluno,8,Rubens Dias


In [5]:
sql_turmas = '''
SELECT * FROM CADASTROS_TURMA
'''
turmas = pd.read_sql(sql_turmas, conn)
turmas

,id,nome,periodo,ano,turno,instituicao_id,usuario_id,curso
0,1,CA32,2,2021,Matutino,1,2,Ciências da Computação
1,2,CAAT32,4,2020,Matutino,1,2,Ciências da Computação
2,3,ES Algoritmos I,1,2011,Noturno,1,2,Engenharia de Software
3,4,CAA55-p,4,2021,Vespertino,2,3,Ciências da Computação


In [13]:
# alunos por turma 
sql_aluno_turma = '''
SELECT T.ID AS id_turma, T.NOME AS turma, UT.USER_ID, P.NOME_COMPLETO
  FROM CADASTROS_TURMA T
       INNER JOIN CADASTROS_USER_TURMA UT ON UT.TURAMA_ID = T.ID 
       INNER JOIN USUARIOS_PERFIL P ON P.ID = UT.USER_ID
-- WHERE T.ID = 4   -- filtro turma
'''
aluno_turma = pd.read_sql(sql_aluno_turma, conn)
aluno_turma = aluno_turma[aluno_turma['id_turma'] == 4]   # filtro turma
aluno_turma

,id_turma,turma,user_id,nome_completo
1,4,CAA55-p,6,Ana Pereira
2,4,CAA55-p,8,Rubens Dias
3,4,CAA55-p,1,Mario Santos
5,4,CAA55-p,5,Roberto Rocha


In [15]:
# turmas professor
sql_turma_prof = '''
SELECT P.ID, P.NOME_COMPLETO,T.ID AS curso_id, T.NOME, T.CURSO 
  FROM USUARIOS_PERFIL P 
       INNER JOIN CADASTROS_TURMA T ON T.USUARIO_ID = P.ID
 --WHERE P.ID = 2  -- filtro professor
'''
turma_prof = pd.read_sql(sql_turma_prof, conn)
turma_prof = turma_prof[turma_prof['id'] == 2]  # filtro professor
turma_prof

,id,nome_completo,curso_id,nome,curso
0,2,Daine Rocha,1,CA32,Ciências da Computação
1,2,Daine Rocha,2,CAAT32,Ciências da Computação
2,2,Daine Rocha,3,ES Algoritmos I,Engenharia de Software


In [21]:
# alunos professor
sql_alunos_prof = '''
WITH PROFESSOR AS (
SELECT UG.ID as prof_id, P.NOME_COMPLETO as professor,T.ID AS curso_id, T.NOME as turma, T.CURSO
  FROM AUTH_GROUP G
       INNER JOIN AUTH_USER_GROUPS UG ON G.ID = UG.GROUP_ID
       INNER JOIN USUARIOS_PERFIL P ON P.ID = UG.ID
       INNER JOIN CADASTROS_TURMA T ON T.USUARIO_ID = P.ID
 WHERE G.ID = 2   -- tras busca apenas professores
  -- AND UG.ID = 2   -- filtro professor
 )
 SELECT P.PROF_ID, P.PROFESSOR, P.CURSO_ID, P.TURMA,P.ID as aluno_id,P.NOME_COMPLETO as aluno  
   FROM PROFESSOR P
        INNER JOIN CADASTROS_USER_TURMA UT ON UT.TURAMA_ID = P.CURSO_ID
        INNER JOIN USUARIOS_PERFIL P ON P.ID = UT.USER_ID
'''
alunos_prof = pd.read_sql(sql_alunos_prof, conn)
alunos_prof = alunos_prof[alunos_prof['prof_id'] == 2]
alunos_prof

,prof_id,professor,curso_id,turma,aluno_id,aluno
0,2,Daine Rocha,1,CA32,1,Mario Santos
4,2,Daine Rocha,2,CAAT32,7,Maria Barros
6,2,Daine Rocha,3,ES Algoritmos I,8,Rubens Dias
7,2,Daine Rocha,2,CAAT32,5,Roberto Rocha


In [26]:
# turmas do aluno
sql_turmas_aluno = '''
SELECT P.ID, P.NOME_COMPLETO,UT.TURAMA_ID,T.NOME
  FROM USUARIOS_PERFIL P
       INNER JOIN CADASTROS_USER_TURMA UT ON UT.USER_ID = P.ID
       INNER JOIN CADASTROS_TURMA T ON T.ID = UT.turama_id
-- WHERE P.ID = 5 -- filtro aluno
'''
turmas_aluno = pd.read_sql(sql_turmas_aluno, conn)
turmas_aluno = turmas_aluno[turmas_aluno['id'] == 5]
turmas_aluno

,id,nome_completo,turama_id,nome
5,5,Roberto Rocha,4,CAA55-p
7,5,Roberto Rocha,2,CAAT32
